<a href="https://colab.research.google.com/github/CarineTalandier/deepLearning/blob/main/projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORT
import numpy as np
import os
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from keras.optimizers import *
from keras import regularizers
from PIL import Image
from tqdm import tqdm
from tensorflow.keras.optimizers import Adama
import time
from keras.models import load_model

In [199]:
def launchConversion(pathData, pathNumpy, resizeImg, imgSize):
    """
    # Permet de lancer la conversion des images en tableau numpy
    :param pathData: chemin ou sont les
    :param pathNumpy:
    :param resizeImg:
    :param imgSize:
    """
    #Pour chaque classe
    for mealClasse in os.listdir(pathData):
      pathMeal = pathData + '/' + mealClasse
      imgs = []

      #Pour chaque image d'une classe, on la charge, resize et transforme en tableau
      for imgMeal in tqdm(os.listdir(pathMeal), "Conversion de la classe : '{}'".format(mealClasse)):
          imgMealPath = pathMeal + '/' + imgMeal
          img = Image.open(imgMealPath)
          img.load()
          if resizeImg == True:
              img = img.resize(size=imgSize)

          data = np.asarray(img, dtype=np.float32)
          imgs.append(data)

      #Converti les gradients de pixels (allant de 0 à 255) vers des gradients compris entre 0 et 1
      imgs = np.asarray(imgs) / 255.

      #Enregistre une classe entiere en un fichier numpy
      np.save(pathNumpy + '/' + mealClasse + '.npy', imgs)

In [200]:
pathNumpy = '/content/numpy'
pathData = '/content/Dataset'
resizeImg = True
imgSize = (50, 50)
launchConversion(pathData, pathNumpy, resizeImg, imgSize)

Conversion de la classe : 'baklava': 100%|██████████| 42/42 [00:00<00:00, 205.56it/s]


In [201]:
def get_labels(path):
    """
    # Permet de recuperer les labels de nos classe, leurs indices dans le tableau et leur matrix binaire one hot encoder
    :param path: chemin ou sont stocké nos fichiers Numpy
    """

    labels = [file.replace('.npy', '') for file in os.listdir(path) if file.endswith('.npy')]
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)

In [220]:
def get_train_test(train_ratio, pathData):
    """
    # Retourner le dataset melanger en dataset d'entrainement et de validation selon un ratio
    :param train_ratio: permet de gerer la part entre dataset de train et de validation
    :param pathData: chemin des fichiers numpy
    """

    labels, _, _ = get_labels(pathData)
    classNumber = 0
 
    #On init avec le premier tableau pour avoir les bonnes dimensions pour la suite
    X = data = np.load(pathData + '/' + labels[0] + '.npy')
    Y = np.zeros(X.shape[0])
    dimension = X[0].shape
    classNumber += 1  

    #On delete le fichier invisible de sauvegarde de googlecolab du tableau des labels
    labels.remove(".ipynb_checkpoints")

    #On ajoute le reste des fichiers numpy de nos classes
    for i, label in enumerate(labels[1:]):
      data = np.load(pathData + '/' + label + '.npy')
      X = np.vstack((X, data))
      Y = np.append(Y, np.full(data.shape[0], fill_value=(i+1)))
      classNumber += 1
  

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=train_ratio)
    return X_train, X_test, to_categorical(Y_train), to_categorical(Y_test), classNumber, dimension

In [221]:
#Definition des chemins et autres variables
pathData = '/content/numpy'
trainRatio = 0.8
epochs = 100
batch_size = 16
earlyStopPatience = 5

#Permet de sauvegarder le model a chaque iteration si il est meilleur que le precedent
#check = ModelCheckpoint('/content/trainedModel/moModel.hdf5', monitor='val_loss', verbose=0,
                        #save_best_only=True, save_weights_only=False, mode='auto')

#Recuperation de nos data pré traité
x_train, x_test, y_train, y_test, classNumber, dimension = get_train_test(trainRatio, pathData)

#On verifie les dimensions de nos données
print('DIMENSION X TRAIN ' + str(x_train.shape))
print('DIMENSION X TEST ' + str(x_test.shape))
print('DIMENSION Y TRAIN ' + str(y_train.shape))
print('DIMENSION Y TEST ' + str(y_test.shape))


#On creer le modele
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(dimension[0], dimension[1], dimension[2])))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(classNumber, activation='softmax'))

#On compile le modele
model.compile(loss='categorical_crossentropy',
                  optimizer=Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999, decay=0.0),
                  metrics=['accuracy'])

#On lance l'entrainement du modele
training = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test))



DIMENSION X TRAIN (163, 50, 50, 3)
DIMENSION X TEST (41, 50, 50, 3)
DIMENSION Y TRAIN (163, 4)
DIMENSION Y TEST (41, 4)
Epoch 1/100
6/6 [==============================] - 1s 123ms/step - loss: 2.1176 - accuracy: 0.2515 - val_loss: 1.3516 - val_accuracy: 0.3415
Epoch 2/100
6/6 [==============================] - 1s 149ms/step - loss: 1.4304 - accuracy: 0.2883 - val_loss: 1.6218 - val_accuracy: 0.2927
Epoch 3/100
6/6 [==============================] - 1s 132ms/step - loss: 1.3418 - accuracy: 0.4233 - val_loss: 1.4820 - val_accuracy: 0.2927
Epoch 4/100
6/6 [==============================] - 1s 185ms/step - loss: 1.2120 - accuracy: 0.4356 - val_loss: 1.2970 - val_accuracy: 0.3171
Epoch 5/100
6/6 [==============================] - 0s 76ms/step - loss: 1.1490 - accuracy: 0.5460 - val_loss: 1.3387 - val_accuracy: 0.2683
Epoch 6/100
6/6 [==============================] - 0s 68ms/step - loss: 1.0821 - accuracy: 0.5644 - val_loss: 1.3730 - val_accuracy: 0.2927
Epoch 7/100
6/6 [===================

PREDICTION

In [222]:
def predict(imagePath, imageSize):
    """
    # Fonction qui permet de convertir une image en array, de charger le modele et de lui injecter notre image pour une prediction
    :param imagePath: chemin de l'image pour realiser une prediction
    :param imageSize: défini la taille de l'image. IMPORTANT : doit être de la même taille que celle des images
    du dataset d'entrainements
    :param label: nom de nos classes de sortie
    """

    start = time.time()

    #Chargement de notre image et traitement
    data = []
    img = Image.open(imagePath)
    img.load()
    img = img.resize(size=imageSize)
    img = np.asarray(img) / 255.
    data.append(img)
    data = np.asarray(data)
  

    #On reshape pour correspondre aux dimensions de notre modele
    # Arg1 : correspond au nombre d'image que on injecte
    # Arg2 : correspond a la largeur de l'image
    # Arg3 : correspond a la hauteur de l'image
    # Arg4 : correspond au nombre de canaux de l'image (1 grayscale, 3 couleurs)
    dimension = data[0].shape

    #Reshape pour passer de 3 à 4 dimension pour notre réseau
    data = data.astype(np.float32).reshape(data.shape[0], dimension[0], dimension[1], dimension[2])
    
    #On realise une prediction
    prediction = model.predict(data)

    #on recupere les labels des classes
    labels, _, _ = get_labels(pathData)

    #On recupere le numero de label qui a la plus haut prediction
    maxPredict = np.argmax(prediction)
   
    #On recupere le mot correspondant à l'indice precedent
    word = labels[maxPredict]  
    pred = prediction[0][maxPredict] * 100.
    end = time.time()


    #On affiche les prédictions
    print()
    print('----------')
    print(" Predictions :")
    for i in range(0, len(labels)):
      if labels[i] != ".ipynb_checkpoints":
        print('     ' + labels[i] + ' : ' + "{0:.2f}%".format(prediction[0][i] * 100.))


    print()
    print('RESULTAT :')
    print('' + word + ' à ' + "{0:.2f}%".format(pred))
    print('temps prediction : ' + "{0:.2f}secs".format(end-start))

    print('----------')

In [225]:
"""
# On definit les chemins d'acces au différentes hyper parametre
"""
imagePath =  '/content/Dataset/applepie/134.jpg'
imageSize = (50,50)

predict(imagePath,imageSize)


----------
 Predictions :
     baklava : 4.98%
     beignets : 2.54%
     applepie : 91.44%
     baby_black_ribs : 1.05%

RESULTAT :
applepie à 91.44%
temps prediction : 0.06secs
----------
